# Acquire

In [3]:
from wrangle_zillow import *
import pandas as pd
pd.options.display.max_rows = 65 

In [4]:
zillow = get_zillow()

In [5]:
encodings, data_dict = get_dict()

In [6]:
df_summary(zillow)

--- Shape: (52292, 61)
--- Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52292 entries, 0 to 52291
Data columns (total 61 columns):
logerror                        52292 non-null float64
transactiondate                 52292 non-null object
id                              52292 non-null int64
parcelid                        52292 non-null int64
airconditioningtypeid           13610 non-null float64
architecturalstyletypeid        70 non-null float64
basementsqft                    47 non-null float64
bathroomcnt                     52292 non-null float64
bedroomcnt                      52292 non-null float64
buildingclasstypeid             0 non-null object
buildingqualitytypeid           33632 non-null float64
calculatedbathnbr               52158 non-null float64
decktypeid                      388 non-null float64
finishedfloor1squarefeet        4368 non-null float64
calculatedfinishedsquarefeet    52211 non-null float64
finishedsquarefeet12            52047 non-null float6

(10554879.015, 26409453.4]    52273
(26409453.4, 42107051.8]          0
(42107051.8, 57804650.2]          0
(57804650.2, 73502248.6]          0
(73502248.6, 89199847.0]          1
(89199847.0, 104897445.4]         0
(104897445.4, 120595043.8]        0
(120595043.8, 136292642.2]        0
(136292642.2, 151990240.6]        0
(151990240.6, 167687839.0]       18
Name: parcelid, dtype: int64
airconditioningtypeid:
1.0     11868
13.0     1567
5.0       159
11.0       16
Name: airconditioningtypeid, dtype: int64
architecturalstyletypeid:
7.0     62
3.0      3
21.0     2
2.0      2
8.0      1
Name: architecturalstyletypeid, dtype: int64
basementsqft:
(34.477000000000004, 390.2]    20
(390.2, 742.4]                 13
(742.4, 1094.6]                 7
(1094.6, 1446.8]                3
(1446.8, 1799.0]                0
(1799.0, 2151.2]                2
(2151.2, 2503.4]                0
(2503.4, 2855.6]                0
(2855.6, 3207.8]                1
(3207.8, 3560.0]                1
Name: base

(3097.9339999999997, 42797.5]    39056
(42797.5, 82104.0]               10580
(82104.0, 121410.5]                824
(121410.5, 160717.0]                 0
(160717.0, 200023.5]                 0
(200023.5, 239330.0]                 0
(239330.0, 278636.5]                17
(278636.5, 317943.0]                 0
(317943.0, 357249.5]                 0
(357249.5, 396556.0]               779
Name: regionidcity, dtype: int64
regionidcounty:
3101.0    33800
1286.0    14110
2061.0     4382
Name: regionidcounty, dtype: int64
regionidneighborhood:
(6194.784, 82673.5]     7253
(82673.5, 158395.0]     2989
(158395.0, 234116.5]      32
(234116.5, 309838.0]    7122
(309838.0, 385559.5]       7
(385559.5, 461281.0]     921
(461281.0, 537002.5]       0
(537002.5, 612724.0]       0
(612724.0, 688445.5]      66
(688445.5, 764167.0]     578
Name: regionidneighborhood, dtype: int64
regionidzip:
(95678.306, 126351.3]    52254
(126351.3, 156720.6]         0
(156720.6, 187089.9]         0
(187089.9, 217459.2

Notes:
- many columns have loads of nulls. Probably going to drop those
- transactiondate is an object instead of some sort of number or date, so it can't sort or display max/min
- There appears to be a 2018 transaction date
- id is totally uninformative
- nulls for basementsqft probably mean no basement
- some of the nulls for AC type id might mean no AC, but there's no way over half the listings lack AC
- someone has a 3200+ sqft basement, 2800+ sqft basement is pretty huge too
- only 12 people with 9+ bathrooms
- 18 with 12+ bedrooms
- buildingclasstypeid doesn't seem particularly useful
- buildingqualitytypeid is really a scale (apparently from 1 to 12??) of how well-built the building is
- decktypeid seems super useless. 614 have type 16 deck which isn't defined anywhere.
- There are several different sqft metrics. calculated sames most useful based on the definitions in data_dict
- some buildings seem to be way too large to be single-unit. There are 23 above 10.7k sqft and 1 over 30k
- there are some really high garagecarcnts
- there are a fair few really large garages
- nulls in hashottuborspa appear to mean that they don't have hot tubs or spas
- holy cannoli there are some massive lot sizes. the 7 with >6M sqft lot size appear to be mislabeled. Perhaps investigate the other high ones later. The 7 houses referenced are pictured below. I believe the entire subdivision lot was listed as the lot size.
- pooltype columns refer to whether the pool is just pool, just hot tub, or both. it's like onehotencoder of pooltype kind of.
- propertylandusecode is county zoning stuff. could be useful in identifying single-unit properties. Probably differs across counties? worth investigating
- If no unit count, drop these propertylandusedescs: residential general, duplex, quadruplex, triplex, cluster home, commerical/office/residential mixed use
- propertyzoningdesc is another zoning column? not sure on the differences between these three
- rawcensustractandblock and regionidcity should probably be categorical, and others involved in this fips categorization
- Very unclear what 0 roomcnt means since they appear to have nonzero bathroomcnt and bedroomcnt. 
- storytypeid relays no information because it only indicates whether a building has a basement, which is already indicated by a non-null value in basementsqft
- really not seeing the value in typeconstructiontypeid. similarly unclear what a null value here indicates
- of course we will drop all unitcnts > 1, and make an educated guess about which null unitcnt properties really have a unitcnt of 1 based on the value_counts within the unitcnt==1 cohort
- there are some strangely large sheds based on yardbuildingsqft. I'm thinking the particularly large ones probably indicate unitcnt > 1 but we shall see
- yearbuilt could influence value. not really going to influence what I drop/don't.
- Six-story house is somehow single-family residential, zoning is unclear but based on comparison of unitcnt == 1 and unitcnt > 1 cohorts, it appears to actually be single-unit. very strange. Latitude, longitude search on google maps shows an elementary school. There is no building in the vicinity that has >1 story.
- fireplaceflag indicates whether or not there is a fireplace at the houses in which fireplacecnt is null.
- probably drop the particularly expensive properties
- null taxdelinquencyflag means their taxes are not past due as of 2015
- taxdelinquencyyear is year from which your delinquent taxes are due. Should be restructured to be a four digit number instead of '99, '00, '01, etc.
- anything with null calculatedfinishedsquarefeet should be dropped.

<img src='huge_lots.png'>

In [7]:
for frame in encodings:
    print(encodings[frame])

    airconditioningtypeid airconditioningdesc
0                       1             Central
1                       2       Chilled Water
2                       3  Evaporative Cooler
3                       4         Geo Thermal
4                       5                None
5                       6               Other
6                       7    Packaged AC Unit
7                       8             Partial
8                       9       Refrigeration
9                      10         Ventilation
10                     11           Wall Unit
11                     12         Window Unit
12                     13                 Yes
    architecturalstyletypeid         architecturalstyledesc
0                          1                        A-Frame
1                          2                       Bungalow
2                          3                       Cape Cod
3                          4                        Cottage
4                          5                       Colon

In [8]:
df_summary(zillow[zillow.unitcnt == 1])

--- Shape: (33734, 61)
--- Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 33734 entries, 3 to 52291
Data columns (total 61 columns):
logerror                        33734 non-null float64
transactiondate                 33734 non-null object
id                              33734 non-null int64
parcelid                        33734 non-null int64
airconditioningtypeid           11549 non-null float64
architecturalstyletypeid        0 non-null float64
basementsqft                    0 non-null float64
bathroomcnt                     33734 non-null float64
bedroomcnt                      33734 non-null float64
buildingclasstypeid             0 non-null object
buildingqualitytypeid           33585 non-null float64
calculatedbathnbr               33713 non-null float64
decktypeid                      0 non-null float64
finishedfloor1squarefeet        0 non-null float64
calculatedfinishedsquarefeet    33733 non-null float64
finishedsquarefeet12            33733 non-null float64
finis

                              num_rows_missing  pct_rows_missing
logerror                                     0          0.000000
transactiondate                              0          0.000000
id                                           0          0.000000
parcelid                                     0          0.000000
airconditioningtypeid                    22185          0.657645
architecturalstyletypeid                 33734          1.000000
basementsqft                             33734          1.000000
bathroomcnt                                  0          0.000000
bedroomcnt                                   0          0.000000
buildingclasstypeid                      33734          1.000000
buildingqualitytypeid                      149          0.004417
calculatedbathnbr                           21          0.000623
decktypeid                               33734          1.000000
finishedfloor1squarefeet                 33734          1.000000
calculatedfinishedsquaref

(-118896316.175, -118770957.6]      89
(-118770957.6, -118646840.2]       335
(-118646840.2, -118522722.8]      3000
(-118522722.8, -118398605.4]      5383
(-118398605.4, -118274488.0]      6713
(-118274488.0, -118150370.6]      6344
(-118150370.6, -118026253.2]      6667
(-118026253.2, -117902135.8]      2552
(-117902135.8, -117778018.4]      1912
(-117778018.4, -117653901.0]       739
Name: longitude, dtype: int64
lotsizesquarefeet:
(-6061.102, 697919.1]     33411
(697919.1, 1394929.2]         6
(1394929.2, 2091939.3]        2
(2091939.3, 2788949.4]        1
(2788949.4, 3485959.5]        1
(3485959.5, 4182969.6]        0
(4182969.6, 4879979.7]        0
(4879979.7, 5576989.8]        0
(5576989.8, 6273999.9]        0
(6273999.9, 6971010.0]        7
Name: lotsizesquarefeet, dtype: int64
poolcnt:
1.0    6832
Name: poolcnt, dtype: int64
poolsizesum:
Series([], Name: poolsizesum, dtype: int64)
pooltypeid10:
Series([], Name: pooltypeid10, dtype: int64)
pooltypeid2:
Series([], Name: pooltype

In [9]:
df_summary(zillow[zillow.unitcnt > 1])

--- Shape: (0, 61)
--- Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 61 columns):
logerror                        0 non-null float64
transactiondate                 0 non-null object
id                              0 non-null int64
parcelid                        0 non-null int64
airconditioningtypeid           0 non-null float64
architecturalstyletypeid        0 non-null float64
basementsqft                    0 non-null float64
bathroomcnt                     0 non-null float64
bedroomcnt                      0 non-null float64
buildingclasstypeid             0 non-null object
buildingqualitytypeid           0 non-null float64
calculatedbathnbr               0 non-null float64
decktypeid                      0 non-null float64
finishedfloor1squarefeet        0 non-null float64
calculatedfinishedsquarefeet    0 non-null float64
finishedsquarefeet12            0 non-null float64
finishedsquarefeet13            0 non-null object
finishedsquarefeet1

Notes to differentiate unitcnt = 1 and other unitcnts
- Decision Tree to differentiate?
- 

In [17]:
data_dict

,Feature,Description
0,'airconditioningtypeid',Type of cooling system present in the home (i...
1,'architecturalstyletypeid',"Architectural style of the home (i.e. ranch, ..."
2,'basementsqft',Finished living area below or partially below...
3,'bathroomcnt',Number of bathrooms in home including fractio...
4,'bedroomcnt',Number of bedrooms in home
5,'buildingqualitytypeid',Overall assessment of condition of the buildi...
6,'buildingclasstypeid',"The building framing type (steel frame, wood f..."
7,'calculatedbathnbr',Number of bathrooms in home including fractio...
8,'decktypeid',Type of deck (if any) present on parcel
9,'threequarterbathnbr',Number of 3/4 bathrooms in house (shower + si...


In [12]:
encodings

{'airconditioningtype':     airconditioningtypeid airconditioningdesc
 0                       1             Central
 1                       2       Chilled Water
 2                       3  Evaporative Cooler
 3                       4         Geo Thermal
 4                       5                None
 5                       6               Other
 6                       7    Packaged AC Unit
 7                       8             Partial
 8                       9       Refrigeration
 9                      10         Ventilation
 10                     11           Wall Unit
 11                     12         Window Unit
 12                     13                 Yes,
 'architecturalstyletype':     architecturalstyletypeid         architecturalstyledesc
 0                          1                        A-Frame
 1                          2                       Bungalow
 2                          3                       Cape Cod
 3                          4                   